In [1]:
!pip install datasets

In [2]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split
from datasets import load_dataset
import kagglehub

In [3]:
from datasets import load_dataset

ds = load_dataset("spikecodes/911-call-transcripts")

README.md:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

calls.jsonl:   0%|          | 0.00/2.08M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/518 [00:00<?, ? examples/s]

In [4]:
ds

DatasetDict({
    train: Dataset({
        features: ['messages'],
        num_rows: 518
    })
})

In [5]:
# Convert the 'train' split to a pandas DataFrame
df = ds['train'].to_pandas()

# Display the DataFrame
print(df)

                                              messages
0    [{'role': 'assistant', 'content': '9-1-1, what...
1    [{'role': 'assistant', 'content': '9-1-1, what...
2    [{'role': 'assistant', 'content': '9-1-1, what...
3    [{'role': 'assistant', 'content': '9-1-1, what...
4    [{'role': 'assistant', 'content': '9-1-1, what...
..                                                 ...
513  [{'role': 'assistant', 'content': '9-1-1, what...
514  [{'role': 'assistant', 'content': '9-1-1, what...
515  [{'role': 'assistant', 'content': '9-1-1, what...
516  [{'role': 'assistant', 'content': '9-1-1, what...
517  [{'role': 'assistant', 'content': '9-1-1, what...

[518 rows x 1 columns]


In [6]:
df

,messages
0,"[{'role': 'assistant', 'content': '9-1-1, what..."
1,"[{'role': 'assistant', 'content': '9-1-1, what..."
2,"[{'role': 'assistant', 'content': '9-1-1, what..."
3,"[{'role': 'assistant', 'content': '9-1-1, what..."
4,"[{'role': 'assistant', 'content': '9-1-1, what..."
...,...
513,"[{'role': 'assistant', 'content': '9-1-1, what..."
514,"[{'role': 'assistant', 'content': '9-1-1, what..."
515,"[{'role': 'assistant', 'content': '9-1-1, what..."
516,"[{'role': 'assistant', 'content': '9-1-1, what..."


In [8]:
df['text'] = df['messages'].apply(lambda x: ' '.join([m['content'] for m in x if m['content'] is not None]))

In [9]:
df

,messages,text
0,"[{'role': 'assistant', 'content': '9-1-1, what...","9-1-1, what's your emergency? I'm at West High..."
1,"[{'role': 'assistant', 'content': '9-1-1, what...","9-1-1, what's your emergency? Somebody just br..."
2,"[{'role': 'assistant', 'content': '9-1-1, what...","9-1-1, what's your emergency? Are you looking,..."
3,"[{'role': 'assistant', 'content': '9-1-1, what...","9-1-1, what's your emergency? Oh, my God. Okay..."
4,"[{'role': 'assistant', 'content': '9-1-1, what...","9-1-1, what's your emergency? Police. Okay, wh..."
...,...,...
513,"[{'role': 'assistant', 'content': '9-1-1, what...","9-1-1, what's your emergency? Can you send a c..."
514,"[{'role': 'assistant', 'content': '9-1-1, what...","9-1-1, what's your emergency? 911, please, fir..."
515,"[{'role': 'assistant', 'content': '9-1-1, what...","9-1-1, what's your emergency? Hey, can you get..."
516,"[{'role': 'assistant', 'content': '9-1-1, what...","9-1-1, what's your emergency? Now we want a ma..."


In [10]:
# Create a list to store the expanded data
expanded_data = []

# Iterate through each row in the original DataFrame
for index, row in df.iterrows():
    for message in row['messages']:
        expanded_data.append({
            'original_index': index,
            'role': message['role'],
            'content': message['content']
        })

# Create the expanded DataFrame from the list of dictionaries
expanded_df = pd.DataFrame(expanded_data)

# Display the first few rows of the new DataFrame
print(expanded_df.head(10))

   original_index       role  \
0               0  assistant   
1               0       user   
2               0  assistant   
3               0       user   
4               0  assistant   
5               0       user   
6               0  assistant   
7               0       user   
8               0  assistant   
9               0       user   

                                             content  
0                      9-1-1, what's your emergency?  
1  I'm at West High School. There's a guy with a ...  
2                                 Which high school?  
3                                         West High.  
4  Okay, we have the police dispatched. Can you g...  
5  I don't know. The guy is just running through ...  
6   Can someone give me a description of the person?  
7      I don't know. Can anybody give a description?  
8  Do we know where in the building? Is he white,...  
9                                      I don't know.  


In [11]:
expanded_df

,original_index,role,content
0,0,assistant,"9-1-1, what's your emergency?"
1,0,user,I'm at West High School. There's a guy with a ...
2,0,assistant,Which high school?
3,0,user,West High.
4,0,assistant,"Okay, we have the police dispatched. Can you g..."
...,...,...,...
25914,517,assistant,Are you on a cordless phone?
25915,517,user,"I have a cordless phone, but I use a walker."
25916,517,assistant,You can go ahead and hang up with me and go ah...
25917,517,user,All right. Bye.


In [12]:
expanded_df['label'] = expanded_df['role'].apply(lambda x: 1 if x == 'assistant' else 0)

In [13]:
expanded_df

,original_index,role,content,label
0,0,assistant,"9-1-1, what's your emergency?",1
1,0,user,I'm at West High School. There's a guy with a ...,0
2,0,assistant,Which high school?,1
3,0,user,West High.,0
4,0,assistant,"Okay, we have the police dispatched. Can you g...",1
...,...,...,...,...
25914,517,assistant,Are you on a cordless phone?,1
25915,517,user,"I have a cordless phone, but I use a walker.",0
25916,517,assistant,You can go ahead and hang up with me and go ah...,1
25917,517,user,All right. Bye.,0


In [14]:
expanded_df['content'].isnull().sum()

1

In [15]:
expanded_df.dropna(inplace=True)

In [16]:
# Rows where content is NaN
nan_rows = expanded_df[expanded_df['content'] == 'NaN']

# Display the rows
print(nan_rows)

Empty DataFrame
Columns: [original_index, role, content, label]
Index: []


In [9]:
expanded_df.set_index('original_index', inplace=True)
expanded_df

,role,content,label
original_index,,,
0,assistant,"9-1-1, what's your emergency?",1
0,user,I'm at West High School. There's a guy with a ...,0
0,assistant,Which high school?,1
0,user,West High.,0
0,assistant,"Okay, we have the police dispatched. Can you g...",1
...,...,...,...
517,assistant,Are you on a cordless phone?,1
517,user,"I have a cordless phone, but I use a walker.",0
517,assistant,You can go ahead and hang up with me and go ah...,1


In [10]:
expanded_df['content'].dropna(inplace=True)

In [11]:
expanded_df['content'].isnull().sum()

11

In [25]:
null_entries = expanded_df[expanded_df['content'].isnull()]
print(null_entries)

Empty DataFrame
Columns: [role, content, label]
Index: []


In [17]:
# Make original_index the index
expanded_df.set_index('original_index', inplace=True)
expanded_df

,role,content,label
original_index,,,
0,assistant,"9-1-1, what's your emergency?",1
0,user,I'm at West High School. There's a guy with a ...,0
0,assistant,Which high school?,1
0,user,West High.,0
0,assistant,"Okay, we have the police dispatched. Can you g...",1
...,...,...,...
517,assistant,Are you on a cordless phone?,1
517,user,"I have a cordless phone, but I use a walker.",0
517,assistant,You can go ahead and hang up with me and go ah...,1


In [18]:
expanded_df.isnull().sum()

role       0
content    0
label      0
dtype: int64

In [19]:
# First look at what we're removing
print("Entries being removed:")
print(expanded_df[expanded_df.index.get_level_values('original_index').isin([78, 451])])

# Then remove them
expanded_df = expanded_df[~expanded_df.index.get_level_values('original_index').isin([78, 451])]

Entries being removed:
                     role                                            content  \
original_index                                                                 
78              assistant                      9-1-1, what's your emergency?   
78                   user                                  1620 Green Place.   
78              assistant                                         1620 what?   
78                   user                                       Green Place.   
78              assistant                 Green Place. Green like the color?   
...                   ...                                                ...   
451                  user                                   It's the police.   
451             assistant  Okay. Step outside and do what they say. Just ...   
451                  user                                      They're here.   
451             assistant  Okay. Just put the phone down and do what they...   
451              

In [20]:
expanded_df.shape

(25799, 3)

In [21]:
expanded_df

,role,content,label
original_index,,,
0,assistant,"9-1-1, what's your emergency?",1
0,user,I'm at West High School. There's a guy with a ...,0
0,assistant,Which high school?,1
0,user,West High.,0
0,assistant,"Okay, we have the police dispatched. Can you g...",1
...,...,...,...
517,assistant,Are you on a cordless phone?,1
517,user,"I have a cordless phone, but I use a walker.",0
517,assistant,You can go ahead and hang up with me and go ah...,1


In [22]:
train_texts, val_texts, train_labels, val_labels = train_test_split(expanded_df['content'].tolist(), expanded_df['label'].tolist(), test_size=0.2, random_state=42)

In [23]:
# Print some statistics
print(f"Total samples: {len(expanded_df)}")
print(f"Training samples: {len(train_texts)}")
print(f"Validation samples: {len(val_texts)}")

Total samples: 25799
Training samples: 20639
Validation samples: 5160


In [24]:
expanded_df

,role,content,label
original_index,,,
0,assistant,"9-1-1, what's your emergency?",1
0,user,I'm at West High School. There's a guy with a ...,0
0,assistant,Which high school?,1
0,user,West High.,0
0,assistant,"Okay, we have the police dispatched. Can you g...",1
...,...,...,...
517,assistant,Are you on a cordless phone?,1
517,user,"I have a cordless phone, but I use a walker.",0
517,assistant,You can go ahead and hang up with me and go ah...,1


In [25]:
class EmergencyCallDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

In [26]:
from transformers import AdamW, get_linear_schedule_with_warmup

# Set up tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [27]:
# Create datasets
train_dataset = EmergencyCallDataset(train_texts, train_labels, tokenizer, max_length=512)
val_dataset = EmergencyCallDataset(val_texts, val_labels, tokenizer, max_length=512)

In [28]:
# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

In [29]:
# Set up optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)
total_steps = len(train_loader) * 10  # 10 epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [30]:
# Set up device (GPU if available, otherwise CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Training loop
for epoch in range(10):
    # Training phase
    model.train()
    for batch in train_loader:
        # Clear previous gradients
        optimizer.zero_grad()
        
        # Move data to device
        inputs = {
            'input_ids': batch['input_ids'].to(device),
            'attention_mask': batch['attention_mask'].to(device),
            'labels': batch['labels'].to(device)
        }
        
        # Forward pass and calculate loss
        outputs = model(**inputs)
        loss = outputs.loss
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        scheduler.step()

    # Validation phase
    model.eval()  # Set model to evaluation mode
    val_loss = 0
    correct = 0
    total = 0
    
    # No gradient calculation needed for validation
    with torch.no_grad():
        for batch in val_loader:
            # Move data to device
            inputs = {
                'input_ids': batch['input_ids'].to(device),
                'attention_mask': batch['attention_mask'].to(device),
                'labels': batch['labels'].to(device)
            }
            
            # Get model predictions
            outputs = model(**inputs)
            
            # Calculate validation loss
            val_loss += outputs.loss.item()
            
            # Calculate accuracy
            _, predictions = torch.max(outputs.logits, 1)
            total += inputs['labels'].size(0)
            correct += (predictions == inputs['labels']).sum().item()
    
    # Print epoch results
    avg_val_loss = val_loss / len(val_loader)
    accuracy = (correct / total) * 100
    print(f'Epoch {epoch+1}:')
    print(f'  Validation Loss: {avg_val_loss:.4f}')
    print(f'  Accuracy: {accuracy:.2f}%')
    print('-' * 50)

Epoch 1:
  Validation Loss: 0.2095
  Accuracy: 93.06%
--------------------------------------------------
Epoch 2:
  Validation Loss: 0.2063
  Accuracy: 92.75%
--------------------------------------------------
Epoch 3:
  Validation Loss: 0.2279
  Accuracy: 93.14%
--------------------------------------------------
Epoch 4:
  Validation Loss: 0.2560
  Accuracy: 92.83%
--------------------------------------------------
Epoch 5:
  Validation Loss: 0.2650
  Accuracy: 92.79%
--------------------------------------------------
Epoch 6:
  Validation Loss: 0.2941
  Accuracy: 93.02%
--------------------------------------------------
Epoch 7:
  Validation Loss: 0.3317
  Accuracy: 92.77%
--------------------------------------------------
Epoch 8:
  Validation Loss: 0.3426
  Accuracy: 92.95%
--------------------------------------------------
Epoch 9:
  Validation Loss: 0.3848
  Accuracy: 92.79%
--------------------------------------------------
Epoch 10:
  Validation Loss: 0.3996
  Accuracy: 92.89%


In [38]:
# Save the fine-tuned model
model.save_pretrained('fine_tuned_bert_emergency_calls')
tokenizer.save_pretrained('fine_tuned_bert_emergency_calls')

('fine_tuned_bert_emergency_calls/tokenizer_config.json',
 'fine_tuned_bert_emergency_calls/special_tokens_map.json',
 'fine_tuned_bert_emergency_calls/vocab.txt',
 'fine_tuned_bert_emergency_calls/added_tokens.json')